<a href="https://colab.research.google.com/github/aaditya9803/ml/blob/main/Sorting_line_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1BZJfMkmr5uyyL1iZGPQen-B3VTYrJjD0'
folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")

    # Get the list of files inside the current folder
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        new_file_name = f"{folder_title}-{index}"
        print(f"Downloading {file_title} as {new_file_name}")

        # Download the file to the current runtime environment
        file.GetContentFile(new_file_name)

Processing folder: niO
Processing folder: biO
Processing folder: riO
Processing folder: wiO


In [2]:
import numpy as np
import pandas as pd
!pip install tsfresh
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
def read_file(filename):
    return pd.read_csv((filename), sep=',')

# index = index - 4
index = 10
white = {}
white_df = pd.DataFrame()
blue = {}
blue_df = pd.DataFrame()
red = {}
red_df = pd.DataFrame()
nio = {}
nio_df = pd.DataFrame()

for j in range (index):
  white[j] = read_file(f"wiO-{j}")
  blue[j] = read_file(f"biO-{j}")
  red[j] = read_file(f"riO-{j}")
  nio[j] = read_file(f"niO-{j}")

In [4]:
df_color = pd.DataFrame()
for i in range(len(white)):
  df_color = pd.concat([df_color, pd.Series(0)])
for i in range(len(blue)):
  df_color = pd.concat([df_color, pd.Series(1)])
for i in range(len(red)):
  df_color = pd.concat([df_color, pd.Series(2)])
for i in range(len(nio)):
  df_color = pd.concat([df_color, pd.Series(3)])
df_color = df_color.astype(int)
df_color.columns = ['color']

In [5]:
df_color.shape

(40, 1)

In [6]:
q= 0
r= index
s = index*2
t = index*3


for j in range(index):
  white[j] = pd.DataFrame(white[j])
  white[j]['fileno'] = q
  white_df = pd.concat([white_df, white[j]])

  blue[j] = pd.DataFrame(blue[j])
  blue[j]['fileno'] = r
  blue_df = pd.concat([blue_df, blue[j]])

  red[j] = pd.DataFrame(red[j])
  red[j]['fileno'] = s
  red_df = pd.concat([red_df, red[j]])

  nio[j] = pd.DataFrame(nio[j])
  nio[j]['fileno'] = t
  nio_df = pd.concat([nio_df, nio[j]])

  q = q + 1
  r = r + 1
  s = s + 1
  t = t + 1


In [7]:
# for i in range (len(white_df)):
#   white_df['fileno'] = i
# for j in range (i, i + len(blue_df)):
#   blue_df['fileno'].append(j)
# for k in range (j,j+len(red_df)):
#   red_df['fileno'].append(j)
# for l in range (k, k+len(nio_df)):
#   nio_df['fileno'].append(j)

In [14]:
white_df

,Datum,Uhrzeit,I_In,V_In,P_In(W),I_Out(A),V_Out(V),P_Out(W),Temp(°C),Energie_In(Wh),Energie_Out(Wh),fileno
0,2024-05-02,2024-05-02 11:37:37.940396:940,0.25,11.65,2.9125,0.29,8.97,2.6013,37.63,0.011648,0.009802,0
1,2024-05-02,2024-05-02 11:37:37.948287:948,0.27,11.52,3.1104,0.25,9.00,2.2500,37.63,0.011654,0.009807,0
2,2024-05-02,2024-05-02 11:37:37.956565:956,0.31,11.65,3.6115,0.27,9.00,2.4300,37.63,0.011663,0.009813,0
3,2024-05-02,2024-05-02 11:37:37.968670:968,0.31,11.65,3.6115,0.25,9.03,2.2575,37.63,0.011675,0.009821,0
4,2024-05-02,2024-05-02 11:37:37.977001:976,0.37,11.65,4.3105,0.33,9.03,2.9799,37.63,0.011685,0.009828,0
...,...,...,...,...,...,...,...,...,...,...,...,...
573,2024-05-02,2024-05-02 11:35:23.762351:762,0.46,11.61,5.3406,0.54,9.00,4.8600,37.38,0.013897,0.007314,9
574,2024-05-02,2024-05-02 11:35:23.774622:774,0.46,11.74,5.4004,0.50,9.00,4.5000,37.63,0.013916,0.007329,9
575,2024-05-02,2024-05-02 11:35:23.782804:782,0.46,11.70,5.3820,0.50,9.03,4.5150,37.38,0.013928,0.007339,9
576,2024-05-02,2024-05-02 11:35:23.794971:794,0.48,11.61,5.5728,0.48,9.03,4.3344,37.63,0.013947,0.007354,9


In [15]:
# white_df ['color'] = 0
# blue_df ['color'] = 1
# red_df ['color'] = 2
# nio_df ['color'] = 3

In [19]:
df = pd.concat([white_df, blue_df, red_df, nio_df])
# for i in range(index*4):
#   df['fileno'] = i
df = df.drop(columns=['Datum'])
x = df.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])
# y = df['color']
# j=0
# y_aggregated = pd.DataFrame()
# for i in range(len(df['fileno'])):
#   if df['fileno'].values[i] ==



  # if df['color'].values[i] == j:
  #   y_aggregated = pd.concat([y_aggregated, pd.Series(j)])
  # elif df['color'].values[i] != j:
  #   j = j + 1
  #   y_aggregated = pd.concat([y_aggregated, pd.Series(j)])
  # else:
  #   print('error')


In [22]:
df.tail()

,Uhrzeit,I_In,V_In,P_In(W),I_Out(A),V_Out(V),P_Out(W),Temp(°C),Energie_In(Wh),Energie_Out(Wh),fileno
885,2024-05-02 13:18:33.353512:353,0.25,11.61,2.9025,0.25,9.00,2.2500,37.88,0.008795,0.006933,39
886,2024-05-02 13:18:33.365492:365,0.29,11.61,3.3669,0.25,9.03,2.2575,38.13,0.008806,0.006940,39
887,2024-05-02 13:18:33.373796:373,0.31,11.61,3.5991,0.31,9.03,2.7993,38.13,0.008814,0.006946,39
888,2024-05-02 13:18:33.385840:385,0.25,11.61,2.9025,0.31,9.03,2.7993,38.13,0.008824,0.006956,39
889,2024-05-02 13:18:33.394443:394,0.27,11.74,3.1698,0.25,9.03,2.2575,38.13,0.008832,0.006961,39


In [23]:
df['fileno'].values

array([ 0,  0,  0, ..., 39, 39, 39])

In [24]:
features =  extract_features(
    x,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)

Feature Extraction: 100%|██████████| 200/200 [02:39<00:00,  1.25it/s]


In [49]:
features.head()

,I_In__variance_larger_than_standard_deviation,I_In__has_duplicate_max,I_In__has_duplicate_min,I_In__has_duplicate,I_In__sum_values,I_In__abs_energy,I_In__mean_abs_change,I_In__mean_change,I_In__mean_second_derivative_central,I_In__median,...,Temp(°C)__fourier_entropy__bins_5,Temp(°C)__fourier_entropy__bins_10,Temp(°C)__fourier_entropy__bins_100,Temp(°C)__permutation_entropy__dimension_3__tau_1,Temp(°C)__permutation_entropy__dimension_4__tau_1,Temp(°C)__permutation_entropy__dimension_5__tau_1,Temp(°C)__permutation_entropy__dimension_6__tau_1,Temp(°C)__permutation_entropy__dimension_7__tau_1,Temp(°C)__query_similarity_count__query_None__threshold_0.0,Temp(°C)__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,1.0,1.0,189.99,60.2121,0.042299,0.000305,-8.051530e-06,0.29,...,1.204209,1.795131,3.805391,1.318114,2.155573,2.989938,3.756227,4.416451,0.0,37.88
1,0.0,0.0,1.0,1.0,185.58,59.6738,0.042040,0.000216,0.000000e+00,0.29,...,1.436031,2.089244,4.045644,1.410305,2.321328,3.249000,4.080992,4.797539,0.0,37.63
2,0.0,0.0,1.0,1.0,184.09,59.0125,0.043344,0.000250,-4.625929e-20,0.30,...,1.139872,1.786103,3.792321,1.462869,2.446170,3.402180,4.301580,5.059894,0.0,37.63
3,0.0,0.0,0.0,1.0,187.40,60.1416,0.042311,0.000246,-1.642036e-05,0.29,...,1.219551,1.839760,3.856714,1.485377,2.535739,3.577661,4.507263,5.212234,0.0,37.63
4,0.0,0.0,1.0,1.0,183.78,59.4834,0.041930,0.000285,1.680672e-05,0.29,...,1.211701,1.898553,3.854095,1.467557,2.476408,3.476928,4.379596,5.122465,0.0,37.63


In [43]:
print(features.shape)
y= df_color['color']
y = y.reset_index(drop=True)
print(y.shape)

(40, 3915)
(40,)


In [44]:

y

,color
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [46]:
selected_features = select_features(features, y)

In [48]:
selected_features.head()

,Temp(°C)__quantile__q_0.9,Temp(°C)__quantile__q_0.1,V_Out(V)__count_above_mean,I_In__number_peaks__n_1,I_Out(A)__count_above_mean,I_In__range_count__max_1000000000000.0__min_0,I_In__range_count__max_1__min_-1,I_Out(A)__length,I_Out(A)__range_count__max_1__min_-1,I_Out(A)__range_count__max_1000000000000.0__min_0,...,"I_Out(A)__fft_coefficient__attr_""abs""__coeff_60","I_In__fft_coefficient__attr_""abs""__coeff_10","I_Out(A)__fft_coefficient__attr_""abs""__coeff_36",V_In__lempel_ziv_complexity__bins_3,V_Out(V)__number_peaks__n_5,I_Out(A)__approximate_entropy__m_2__r_0.3,"I_In__fft_coefficient__attr_""imag""__coeff_5","I_Out(A)__fft_coefficient__attr_""imag""__coeff_5",I_In__energy_ratio_by_chunks__num_segments_10__segment_focus_5,Temp(°C)__count_above_mean
0,37.63,37.38,241.0,148.0,201.0,623.0,623.0,623.0,622.0,623.0,...,0.465958,2.676195,1.059637,0.215088,21.0,0.954003,4.069768,4.690881,0.089467,444.0
1,37.63,37.38,244.0,135.0,214.0,604.0,603.0,604.0,602.0,604.0,...,0.887005,3.793014,1.994054,0.218543,10.0,0.970813,4.447113,5.643517,0.088622,350.0
2,37.63,37.38,237.0,136.0,207.0,602.0,601.0,602.0,601.0,602.0,...,1.077024,4.448517,2.564212,0.219269,10.0,0.968433,3.983641,4.614474,0.088571,284.0
3,37.63,37.38,231.0,146.0,189.0,611.0,610.0,611.0,610.0,611.0,...,1.217607,3.043308,2.246537,0.211129,12.0,0.943778,3.635062,5.929733,0.088666,247.0
4,37.63,37.38,238.0,122.0,200.0,597.0,596.0,597.0,596.0,597.0,...,0.427673,4.116952,2.441465,0.212730,12.0,0.987570,3.490579,4.509793,0.086189,256.0
